## Notebook for Production of DE1 tables
Intro

### Import packages & functions

In [1]:
library(haven)
library(PxWebApiData)
library(tidyverse)
library(klassR)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
# define paths

functions_path <- paste(getwd(), "functions/", sep = "/")

# source function and correspondence

source(paste(functions_path, "functions.r", sep = "/"))
source(paste(functions_path, "correspondence.r", sep = "/"))

### Statbank data import

In [21]:
# import data for population variables

df_DE1 <- ApiData(07459,
                  Region = c("0301", "1103", "4601", "5001", "3005"),
                  Alder = list("agg:FemAarigGruppering",
                               c("F00-04", "F05-09", "F10-14",
                                 "F15-19", "F20-24", "F25-29",
                                 "F30-34", "F35-39", "F40-44",
                                 "F45-49", "F50-54", "F55-59",
                                 "F60-64", "F65-69", "F70-74",
                                 "F75-79", "F80-84", "F85-89",
                                 "F90-94", "F95-99", "F100G5+")),
                  Tid = "2022")[[2]]

In [22]:
# import data for median

df_DE1073V <- ApiData(13536,
                      Region = list("vs:Kommune", c("0301",
                                                    "1103",
                                                    "4601",
                                                    "5001",
                                                    "3005")),
                      ContentsCode = "Medianalder",
                      Tid = "2022")[[2]]

### Data processing
1. Add a column with municipality names onto the dataframe based on region code (klass = 131)
2. Load a correspondence table between city name and city code
3. Merge the dataframe and city correspondence 

In [23]:
# import klass correspondence for city names

df_DE1$name <- ApplyKlass(df_DE1$Region, klass = 131)
df_DE1073V$name <- ApplyKlass(df_DE1073V$Region, klass = 131)

city_correspondence <- GetKlass(550) %>%
                           select(code, name)

In [24]:
# merge klass correspondence with dataframe

df_DE1 <- merge.data.frame(df_DE1, city_correspondence,
                           by = "name")

df_DE1073V <- merge.data.frame(df_DE1073V, city_correspondence,
                               by = "name")

### Summary tables

#### Part 1:
This section of code produces tables that require summing over several rows of the imported age and gender separated data. In particular, 
  
DE1001V - DE1003V: Population on the 1st of January (total, male and female);  
DE1073V: Median population age;  
DE1137V - DE1139V: Population on the 1st of January, 85 years or over.

In [7]:
# DE1001V - DE1003V

DE1001V_2022 <- df_DE1 %>%
                   group_by(code) %>%
                   summarise(Value = sum(value))

DE1002V_2022 <- sum_rows_DE1(Kjonn, "1")

DE1003V_2022 <- sum_rows_DE1(Kjonn, "2")

In [8]:
# DE1073V

DE10073_2022 <- df_DE1073V %>%
                    select(code, value) %>%
                    rename(Value = value)

In [9]:
# DE1137V - DE1139V

DE1137V_2022 <- df_DE1 %>%
                    filter(Alder %in% c("F85-89",
                                        "F90-94",
                                        "F95-99",
                                        "F100G5+")) %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

DE1138V_2022 <- df_DE1 %>%
                    filter(Alder %in% c("F85-89",
                                        "F90-94",
                                        "F95-99",
                                        "F100G5+"),
                           Kjonn == "1") %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

DE1139V_2022 <- df_DE1 %>%
                    filter(Alder %in% c("F85-89",
                                        "F90-94",
                                        "F95-99",
                                        "F100G5+"),
                          Kjonn == '2') %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

#### Part 2.1 Totals:
This section of code loops over the variables defined in correspondence.r and creates all 'total' tables. 

In [10]:
# loop over variable_names with totals and create summary tables using sum_rows

for (i in seq(1, nrow(variable_correspondence), 3)) {
    assign(
        paste(as.character(variable_correspondence$var_name[i]),
              "2022",
              sep = "_"),
        sum_rows_DE1(Alder, as.character(variable_correspondence$var_age[i])))
}

#### Part 2.2. Gender separated tables
Note, for the gender seperated tables we do not have to sum over rows, but simply filter at the age group and gender. This is done using the function: filter_rows, and specifying *age_group* as the first variable and *gender_code* as the second.

In [11]:
# set index to be all values that are not totals

index <- setdiff(1:nrow(variable_correspondence),
                 seq(1, nrow(variable_correspondence), 3))

In [12]:
# loop over index and create summary tables using filter_rows

for (i in index) {
    assign(
        paste(as.character(variable_correspondence$var_name[i]),
              "2022",
              sep = "_"),
        filter_rows_DE1(as.character(variable_correspondence$var_age[i]),
                    as.character(variable_correspondence$var_gender[i])))
}

### Table formatting

In [13]:
my_list <- ls(pattern = "^[A-Z]{2}.*_2022")

In [14]:
for (table in my_list) {
    assign(paste(table), table_formatting(table))
}

In [15]:
ls(pattern = "^[A-Z]{2}.*_2022")

[1] "DE1001V_2022" "DE1002V_2022" "DE1003V_2022" "DE10073_2022" "DE1040V_2022"
 [6] "DE1041V_2022" "DE1042V_2022" "DE1046V_2022" "DE1047V_2022" "DE1048V_2022"
[11] "DE1049V_2022" "DE1050V_2022" "DE1051V_2022" "DE1074V_2022" "DE1075V_2022"
[16] "DE1076V_2022" "DE1077V_2022" "DE1078V_2022" "DE1079V_2022" "DE1101V_2022"
[21] "DE1102V_2022" "DE1103V_2022" "DE1104V_2022" "DE1105V_2022" "DE1106V_2022"
[26] "DE1107V_2022" "DE1108V_2022" "DE1109V_2022" "DE1110V_2022" "DE1111V_2022"
[31] "DE1112V_2022" "DE1113V_2022" "DE1114V_2022" "DE1115V_2022" "DE1116V_2022"
[36] "DE1117V_2022" "DE1118V_2022" "DE1119V_2022" "DE1120V_2022" "DE1121V_2022"
[41] "DE1122V_2022" "DE1123V_2022" "DE1124V_2022" "DE1125V_2022" "DE1126V_2022"
[46] "DE1127V_2022" "DE1128V_2022" "DE1129V_2022" "DE1130V_2022" "DE1131V_2022"
[51] "DE1132V_2022" "DE1133V_2022" "DE1134V_2022" "DE1135V_2022" "DE1136V_2022"
[56] "DE1137V_2022" "DE1138V_2022" "DE1139V_2022"

In [20]:
DE1114V_2022

City_code,Variable_code,Reference_year,Value,Flags,Footnote
<chr>,<chr>,<chr>,<int>,<chr>,<chr>
NO001C,DE1114V,2022,24327,,
NO002C,DE1114V,2022,5141,,
NO003C,DE1114V,2022,9798,,
NO004C,DE1114V,2022,6888,,
NO005C,DE1114V,2022,3790,,


## Export
Add code for exporting to csv (this is the current solution for the presentation).

In [17]:
# create an empty list to fill in subsequent loop

DE_tables <- list()

# looping over variables/dfs specified by pattern, before appending to list

for (i in ls(pattern = "^DE.*2022")) {
    x <- get(i)
    DE_tables[[i]] <- x
    }

In [18]:
# rbind the tables

DE_2022 <- do.call("rbind", DE_tables)

In [19]:
# export to Linux

# write.csv(DE_2022, "/ssb/stamme01/bediv/project/CITYSTATISTICS/DE_2022.csv", row.names = FALSE)